In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
from tensorflow import keras

In [2]:
EMBEDDING_LAYER_URL = 'https://tfhub.dev/google/nnlm-en-dim50/2'

In [3]:
BATCH_SIZE = 512
SHUFFLE_BUFFER_SIZE = 10000

In [4]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)
train_data, val_data = imdb['train'], imdb['test']

In [5]:
# (train_data, val_data, test_data), info = tfds.load(
#     name='imdb_reviews', 
#     with_info=True,
#     split=('train[:60%]', 'train[60%:]', 'test'),
#     as_supervised=True
# )

In [6]:
train_ds = train_data\
    .shuffle(SHUFFLE_BUFFER_SIZE)\
    .batch(BATCH_SIZE)

val_ds = val_data.batch(BATCH_SIZE)

In [7]:
embedding_layer = hub.KerasLayer(EMBEDDING_LAYER_URL, input_shape=[], dtype=tf.string, trainable=True)

In [8]:
model = keras.Sequential([
    embedding_layer,
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

model.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    mode='min',
    # restore_best_weights=True
)

history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=[early_stopping],
    verbose=1
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 16)                816       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
49/49 [==============================] - 6s 101ms/step - loss: 0.6665 - accuracy: 0.5864 - val_loss: 0.5418 - val_accuracy: 0.7699
Epoch 2/10
49/49 [==============================] - 5s 95ms/step - loss: 0.4770 - accuracy: 0.8174 - val_loss: 0.3996 - val_accuracy: 0.8364
Epoch 3/10
49/49 [==============================] - 5s 98ms/step - loss: 0.3173 - accuracy:

In [9]:
model.evaluate(val_ds)

49/49 [==============================] - 2s 39ms/step - loss: 0.3606 - accuracy: 0.8589


[0.36059218645095825, 0.8588799834251404]

In [10]:
new_data = "There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come."
prediction = model.predict([new_data])
prediction

array([[0.626334]], dtype=float32)